In [2]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os
from io import StringIO
from Tools import *
import csv
from Tools import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [3]:
# 讀取INI檔案
InitFile = IniTool()
# 讀取路徑
DataPath = InitFile.Read('Infomation', 'DataPath')
# 總資料開始日期
DataStartDate = InitFile.Read('Infomation', 'DataStartDate')
# 開始統計的日期
StartDate = InitFile.Read('Infomation', 'StartDate')


Load init file: .\Initialization.ini


In [4]:
def GetDataFrame_(date):
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL';
    print (url)
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL')
    # 整理一些字串：
#     df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    return df

In [190]:
def GetDataFrame(date):
    file = DataPath + date + '.csv'
    datas = pd.DataFrame()
#     print(file)
#     with open(file) as csvFile:
#         csvReader = csv.reader(csvFile)
#         datas = list(csvReader)
        
    f = open(file, 'r')
    text = f.read()
    f.close()
#     for line in f.readlines():
#         print(line)

    
#     print(datas[0])
    
    df = pd.read_csv(StringIO(text.replace("=", "")), thousands=",",
        header=["證券代號" in l for l in text.split("\n")].index(True)-1)
    df.insert(0,'Date',date)
#     new_header = datas.loc['證券代號']
#     new_header = create_new_header(new_header)
#     df = datas.loc['證券代號']
    return df
# GetDataFrame('20210804')

In [191]:
#GetDataFrame('20210804')

In [192]:
# 字串轉Datetime
begin = datetime.strptime(StartDate, "%Y-%m-%d")
begin

datetime.datetime(2020, 1, 1, 0, 0)

In [260]:
end = datetime.today() - timedelta(days=7)
end = datetime.today()

In [250]:
# 讀取休市清單
df_HS = pd.read_csv('holidaySchedule.csv', index_col = 0)
df_HS.loc[:,'Date'].min()

20190101

In [261]:
# 建立搜索清單
DayList = []
for i in range((end - begin).days+1):
    day = begin + timedelta(days=i)
    w_day = day.weekday()
    c_day = day.strftime("%Y%m%d")
    NotInList = True
    for index, row in df_HS.iterrows():
        if str(c_day) == str(row['Date']):
            NotInList = False
    if (w_day != 5 and w_day !=6 and NotInList):
        DayList.append(c_day)

In [262]:
ListLen = len(DayList)
ListLen

399

In [253]:
StartTime = dtime.now()
for idx, val in enumerate(DayList):
    CurrentTime = dtime.now()
    if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
        break;
    
    fullpath = DataPath + val + '.csv'
    # 確認是否有舊資料
    NoHistory = False
    if os.path.isfile(fullpath):
        # 檔案存在的處理
        print("\r 檔案存在。" + str(idx), end="")
    else:
        # 檔案不存在的處理
        print("\n檔案不存在。")
        NoHistory = True
    if NoHistory is True:
        url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + val + '&type=ALL'
        print ("\r Loading... "+ str(round((idx+1)/ListLen*100,2)) + "% " +val + " ", end="")
        r = requests.get(url, allow_redirects=True)
        open(fullpath, 'wb').write(r.content)
        time.sleep(15)
    


 檔案存在。393

In [238]:
test1 = GetDataFrame('20210803')
fliter = (test1['證券代號'] == '3034')
test1[fliter]

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
21856,20210803,3034,聯詠,12051832,12791,6593439179,544.00,555.00,541.00,545.00,+,4.0,545.00,1.0,546.00,63.0,14.6,NaN


In [254]:
# 建立特定股票的清單
List = []
DFList = list()
for idx, val in enumerate(DayList):
    CurrentTime = dtime.now()
    if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
        break;
    
    fullpath = DataPath + val + '.csv'
    # 確認是否有舊資料
    NoHistory = False
    if os.path.isfile(fullpath):
        # 檔案存在的處理
        print("\r 檔案存在。" + str(idx) + " " + val, end="")
    else:
        # 檔案不存在的處理
        print("\n檔案不存在。")
        NoHistory = True
    if NoHistory is False:
#         print(val)
        df = GetDataFrame(val)
        DFList.append(df)
#         print(df)
#         fliter = (df['證券代號'] == '3034')

 檔案存在。393 20210819

In [263]:
# 更新清單
def UpdateDFList():
    CurrentTime = dtime.now()
    Start = False
    LastDataDay = DFList[-1].loc[0,'Date']
    for idx, val in enumerate(DayList):

        if LastDataDay == val:
            Start = True
            print(idx)
            continue
        if Start is True:
#             print ("WTF")
            if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
                break;
            fullpath = DataPath + val + '.csv'
            # 確認是否有舊資料
            NoHistory = False
            if os.path.isfile(fullpath):
                # 檔案存在的處理
                print("\r 檔案存在。" + str(idx) + " " + val, end="")
            else:
                # 檔案不存在的處理
                print("\n檔案不存在。")
                NoHistory = True
            if NoHistory is False:
    #             print(val)
                df = GetDataFrame(val)
                DFList.append(df)
        
UpdateDFList()

393
WTF
 檔案存在。394 20210820WTF
 檔案存在。395 20210823WTF
 檔案存在。396 20210824WTF
 檔案存在。397 20210825WTF


In [264]:
DFList[-1]

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,20210825,0050,元大台灣50,7031858,6772,954442277,135.15,136.10,135.15,136.05,+,1.30,136.05,213,136.10,26,0.00,NaN
1,20210825,0051,元大中型100,37658,52,2118050,56.00,56.30,56.00,56.30,+,0.45,56.25,1,56.30,10,0.00,NaN
2,20210825,0052,富邦科技,710596,243,87181997,122.00,123.00,121.85,122.95,+,1.70,122.95,25,123.00,17,0.00,NaN
3,20210825,0053,元大電子,1000,1,65750,65.75,65.75,65.75,65.75,+,0.65,65.75,1,65.80,3,0.00,NaN
4,20210825,0054,元大台商50,2473,9,76318,31.07,31.07,31.07,31.07,+,0.35,30.99,1,31.08,1,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22710,20210825,9944,新麗,94086,66,1911852,20.40,20.40,20.10,20.20,+,0.05,20.20,1,20.35,1,22.20,NaN
22711,20210825,9945,潤泰新,3457248,2063,225553072,64.90,65.70,64.70,65.40,+,0.60,65.40,5,65.50,372,6.75,NaN
22712,20210825,9946,三發地產,289105,112,4778785,16.70,16.70,16.45,16.50,,0.00,16.50,58,16.55,1,75.00,NaN
22713,20210825,9955,佳龍,86504,88,1489883,17.10,17.30,17.10,17.25,+,0.20,17.25,2,17.30,4,0.00,NaN


In [200]:
SelectDataList = pd.DataFrame()
for dt in DFList:
#     fliter = (dt['證券代號'] == '3034')
    if SelectDataList.empty:
        SelectDataList = dt.loc[dt['證券代號'] == '1235']
    else:
        SelectDataList = pd.concat([SelectDataList, dt.loc[dt['證券代號'] == '1235']], ignore_index=True )
#     print(dt.loc[dt['證券代號'] == '3034'], ignore_index=True)
SelectDataList

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,20200103,1235,興泰,82198,71,1804656,21.90,22.00,21.90,22.00,+,0.20,21.95,1.0,22.00,12.0,366.67,NaN
1,20200106,1235,興泰,30552,31,669944,21.95,22.00,21.85,22.00,,0.00,21.95,2.0,22.00,3.0,366.67,NaN
2,20200107,1235,興泰,27366,28,600938,21.95,22.10,21.90,22.10,+,0.10,22.05,1.0,22.10,6.0,368.33,NaN
3,20200108,1235,興泰,30500,23,670650,22.05,22.05,21.95,22.00,-,0.10,21.85,3.0,22.00,3.0,366.67,NaN
4,20200109,1235,興泰,515260,13,11184420,22.00,22.00,21.80,22.00,,0.00,21.90,4.0,22.00,7.0,366.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,20210818,1235,興泰,2005,3,68867,34.35,34.35,34.35,34.35,,0.00,33.50,1.0,34.10,6.0,81.79,NaN
393,20210819,1235,興泰,56145,37,1854057,33.35,33.90,32.60,33.90,-,0.45,33.80,1.0,33.90,2.0,80.71,NaN
394,20210820,1235,興泰,16834,15,559063,33.15,33.85,33.00,33.85,-,0.05,33.50,2.0,33.85,1.0,80.60,NaN
395,20210823,1235,興泰,40069,34,1337284,33.85,34.00,33.05,34.00,+,0.15,33.85,1.0,34.00,6.0,80.95,NaN


In [201]:
def GetSelectDataList(src, code):
    dst = pd.DataFrame()
    for dt in src:
        if dst.empty:
            dst = dt.loc[dt['證券代號'] == code]
        else:
            dst = pd.concat([dst, dt.loc[dt['證券代號'] == code]], ignore_index=True )
    return dst

In [202]:
SelectDataList = GetSelectDataList(DFList, '')
SelectDataList

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16


In [203]:
# 往前查找
def GetPricePrevious(data, index):
    print('GetPricePrevious')
    if index == 0:
        return -1
    else:
        currentIndex = index - 1
        price = data.loc[current_index, '收盤價']
        if price == '--':
            price = GetPricePrevious(data, currentIndex)
        else:
            return price
    
# 往後查找
def GetPriceNext(data, index):
    print('GetPriceNext')
    DataLen = len(data)
    if index + 1 >= DataLen:
        return -1
    else:
        currentIndex = index + 1
        price = data.loc[currentIndex, '收盤價']
        change = data.loc[currentIndex, '漲跌價差']
        new_price = -1
        if price == '--':
            new_price = GetPriceNext(data, currentIndex)
        else:
            new_price = str(float(price) - change)
        return new_price
    
def GetPrice(data, index):
    # 防呆
    if data.loc[index, '收盤價'] != '--':
        return data.loc[index, '收盤價']
    else:
        price = GetPricePrevious(data, index)
        if price == -1:
            # 往前找沒找到的情況往後找
            price = GetPriceNext(data, index)
        return price



In [19]:
price = GetPriceNext(SelectDataList, 87)
print(price)
print(SelectDataList.loc[87,'收盤價'])
print(SelectDataList.loc[87,'漲跌價差'])
print(SelectDataList.loc[88,'收盤價'])
print(SelectDataList.loc[88,'漲跌價差'])
df = pd.DataFrame()
df = SelectDataList.loc[86]
print(df)
df = pd.concat([df, SelectDataList.loc[87]], ignore_index=False )
df = pd.concat([df, SelectDataList.loc[88]], ignore_index=False )
df

# pPrice = GetPrice(SelectDataList, 87)
# print(SelectDataList.loc[86,'收盤價'])
# print(SelectDataList.loc[87,'漲跌價差'])
# print(pPrice)
# SelectDataList.loc[87,'收盤價'] = str(pPrice)
# print(SelectDataList.loc[87,'收盤價'])

GetPriceNext
-1


KeyError: 87

In [204]:
def CheckList(code, codelist):
    foundit = False
    for c in codelist:
        if c == code:
            foundit = True
            break
    return foundit

def BuildListByDeal( datalist, lower, day):
    CodeList = []
    count = day
    for idx, dayData in enumerate(reversed(DFList)):
        
        check = False
#         print(dayData)
#         for idy, stockData in enumerate(dayData): 
#         print (dayData)
#         print (len(dayData))
        datacount = 0
        datalen = len(dayData)
#        print (count)
        print("\r" + str(count) + ' ', end="")
        for idy in range(datalen) :
            #print(idy)
            deal = int(dayData.loc[idy,'成交筆數'])
            code = dayData.loc[idy,'證券代號']
            #print (code + ' ' + str(deal))
            #deal = stockData.loc['成交筆數']
#             print(stockData)
#             deal = 0
            
            if deal > lower:
                check = True
                if CheckList(code, CodeList) is False:
                    CodeList.append(code)
        count -= 1
        if count == 0:
            break
    return CodeList
CodeList = BuildListByDeal(DFList, 1000, 10)

print(CodeList)

1  ['0050', '0056', '006208', '00631L', '00632R', '00633L', '00637L', '00642U', '00715L', '00878', '00881', '00882', '00885', '00891', '00892', '00893', '00894', '00895', '1101', '1102', '1210', '1216', '1301', '1303', '1304', '1305', '1308', '1309', '1310', '1312', '1313', '1314', '1326', '1402', '1409', '1440', '1455', '1504', '1513', '1517', '1590', '1605', '1608', '1609', '1708', '1714', '1717', '1718', '1722', '1727', '1734', '1783', '1795', '1802', '1903', '1904', '1905', '1907', '1909', '2002', '2006', '2007', '2009', '2010', '2013', '2014', '2015', '2017', '2020', '2022', '2023', '2027', '2028', '2029', '2030', '2031', '2032', '2034', '2038', '2069', '2102', '2103', '2105', '2106', '2108', '2208', '2301', '2303', '2308', '2312', '2313', '2314', '2317', '2323', '2324', '2327', '2328', '2329', '2330', '2331', '2337', '2338', '2340', '2342', '2344', '2345', '2351', '2352', '2353', '2354', '2356', '2357', '2358', '2363', '2367', '2368', '2369', '2371', '2375', '2376', '2377', '2379

##### for idx, dayData in enumerate(DFList):
    for idy, price in reversed(list(enumerate(dt['收盤價']))):
        if price == '--':
            print(str(idx) + ' ' + str(idy))
            

In [205]:
len(CodeList)

470

In [206]:
# 依據CodeList建立表格
def GetPriceTable(datalist, codelist):
    PriceDFs = []
    Len = len(CodeList)
    for idy, code in enumerate(codelist):
        print("\r" + str(idy) + '/' + str(Len), end="")
        SelectDataList = GetSelectDataList(datalist, code)
        PriceDFs.append(SelectDataList)
#     print(PriceDF)
#     for idx, dayData in enumerate(datalist):
#         print(idx)
#         for idy, code in enumerate(codelist):
#             price = dayData.loc[idy, '收盤價']
# #             print (price)
#             if price == '--':
#                 print(str(idx) + ' ' + str(idy))
#                 SelectDataList = GetSelectDataList(datalist, code)
#                 print(SelectDataList)
#                 price = GetPricePrevious(SelectDataList,idx)
# #             print(price)
#             PriceDF.iloc[idy,idx] = price
# #         print(PriceDF)
    return PriceDFs
    
PriceDFs = GetPriceTable(DFList,CodeList)


469/470

In [211]:
CodeDataPath = "./CodeData/"
for idx, code in enumerate(CodeList):
    path = CodeDataPath + code + ".csv"
    PriceDFs[idx].to_csv(path, encoding='utf_8_sig')
    

In [189]:
# PriceDFs[0]
def GetMA(datalist, index):
    List = []
    df = datalist[index]
#     print(df.loc[1])
    print(len(df))
    days = [5,10,20,60,200]
    ma = [0,0,0,0,0]
    total = 0
    count = 0;
    for idx in reversed(range(len(df))):
        count += 1;
        price = df.loc[idx,'收盤價']
        total += float(price)
        if count == 5:
            ma[0] = total / count
        if count == 10:
            ma[1] = total / count
        if count == 20:
            ma[2] = total / count
        if count == 60:
            ma[3] = total / count
        if count == 200:
            ma[4] = total / count
    return ma
            
MA = GetMA(PriceDFs, 3)
print (MA)

395
[119.41, 123.69500000000001, 126.86500000000001, 128.10249999999996, 108.69]


In [164]:
# def GetMA(datalist, code, days):


#     List = []
#     count = days
#     dst = pd.DataFrame()
#     for idx, dayData in enumerate(reversed(DFList)):
        
# #         dst = dayData.loc[dayData['證券代號'] == code]
# #         print(dst)
#         for idy in range(len(dayData)):
#             if code == dayData.loc[idy, '證券代號']:
#                 price = dayData.loc[idy,'收盤價']
#                 if price == '--':
#                     SelectDataList = GetSelectDataList(datalist, code)
#                     GetPricePrevious(SelectDataList,code)
#                 print (dayData.loc[idy,'收盤價'])
#         count -= 1
#         if count <= 0:
#             break
#     return dst
# #    for idx, price in enumerate(data['收盤價']):
# #         print(str(idx)+" "+price)
# #        if price == '--':
# #            print(idx)
# #     for dt in data:
# #           if dt['收盤價'].contains('--'):
# #                 List.append(dt)
# Result = GetMA(DFList, '3006', 5)
# Result